__COMSW4121 Assignment 3: 
Spark Machine Learning Assignment (from Chapter 3 of Advanced Analytics using Spark book)
__ 


Patrick Rogan

UNI: psr2125

Collaborator: Joshua Safyan (jds2258)

In [1]:
sc

In [1]:
# read in matrix of user-artist data, from readme:
# user_artist_data.txt
#    3 columns: userid artistid playcount

rawUserArtistData = sc.textFile("audio_data/user_artist_data_r.txt")

NameError: name 'sc' is not defined

In [3]:
# Read in map between artist name and id, create map. If there is no artist id, map to (None, None)
# artist_data.txt
#    2 columns: artistid artist_name

rawArtistData = sc.textFile("audio_data/artist_data.txt")

def parseArtist(lines):
    line = lines.split('\t')
    try: 
        int(line[0])
        line[1] = " ".join(line[1:])  # join every list element from after the first tab split
        return [(int(line[0]), line[1].encode('ascii','ignore').strip())] # strip out unicode characters
    except:
        return [(None,None)]
    
artistByID = rawArtistData.flatMap(parseArtist)

In [4]:
# read in matrix that maps misspelled or nonstandard names to the artist’s canonical name, 
# as with artist id, if the line is corrupted, map to (None, None) then collect these as a map.
# from readme:
# artist_alias.txt
#    2 columns: badid, goodid
#    known incorrectly spelt artists and the correct artist id. 
#    you can correct errors in user_artist_data as you read it in using this file
#    (we're not yet finished merging this data)

rawArtistAlias = sc.textFile("audio_data/artist_alias.txt")

def splitNclean(lines):
    if lines.split('\t')[0]:
        return [(int(lines.split('\t')[0]), int(lines.split('\t')[1]))]
    else:
        return [(None,None)]

artistAlias = rawArtistAlias.flatMap(splitNclean).collectAsMap()

In [5]:
# Import ALS and other modules
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [6]:
# Map all artist ids in the rawUserArtistData to their canonical ids and 
# create a Ratings object. 
# 
# After completing this step, fit the model on the data using predefined
# hyperparameters.

bArtistAlias = sc.broadcast(artistAlias)

def processToRating(lines):
    fields = [int(x) for x in lines.split(' ')]
    
    # lookup canonical artist id, if it does not exist use default
    finalArtistID = bArtistAlias.value.get(fields[1])
    if not finalArtistID:
        finalArtistID = fields[1]

    # (userID, artistID, count)
    return Rating(fields[0],finalArtistID,fields[2])

trainData = rawUserArtistData.map(processToRating).cache()

model = ALS.trainImplicit(trainData, 10, 5, 0.01)

In [7]:
# Print top 10 recommendations for a user from the fitted model

testUserID = 2093760

recommendations = model.call("recommendProducts", testUserID, 10)

for rec in recommendations: 
    print rec

Rating(user=2093760, product=1007614, rating=0.032505367281193485)
Rating(user=2093760, product=4605, rating=0.030709324769752626)
Rating(user=2093760, product=2814, rating=0.03035657717182504)
Rating(user=2093760, product=1001819, rating=0.02904980509991882)
Rating(user=2093760, product=829, rating=0.028654810237964203)
Rating(user=2093760, product=1037970, rating=0.027949257190184897)
Rating(user=2093760, product=1811, rating=0.027948151817487707)
Rating(user=2093760, product=250, rating=0.027449306475440555)
Rating(user=2093760, product=930, rating=0.02636111017375321)
Rating(user=2093760, product=1003249, rating=0.024860725307017797)


In [8]:
# Print the corresponding canonical artist name for the top 10 recommendations

for rec in recommendations:
    print artistByID.lookup(rec[1])

['Jay-Z']
['Snoop Dogg']
['50 Cent']
['2Pac']
['Nas']
['Kanye West']
['Dr. Dre']
['Outkast']
['Eminem']
['Ludacris']
